### [DATASET LINK](https://drive.google.com/file/d/1lWgKokYUrD5PPMO3tCy-yMN2ytaSnjTV/view?usp=sharing)

### First Approach
- Extracting all the Features Like histogram_equalization, grayscale_transformation, edge_detection, hough_transform
- Passing it to PCA to reduce the Dimensions
-  Training a model and checking its Accuracy

In [145]:
import os
import cv2
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

In [146]:
# Define paths to datasets
dataset_paths = os.getcwd() + "/dataset_1/dataset_full"

dataset_paths = {
    "Building": os.path.join(dataset_paths, "Building"),
    "Forest": os.path.join(dataset_paths, "Forest"),
    "Glacier": os.path.join(dataset_paths, "Glacier"),
    "Mountain": os.path.join(dataset_paths, "Mountains"),
    "Sea": os.path.join(dataset_paths, "Sea"),
    "Street": os.path.join(dataset_paths, "Streets")
}

In [147]:
# Feature extraction functions

def histogram_equalization(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    equalized = cv2.equalizeHist(gray)
    return equalized


def grayscale_transformation(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


def image_smoothing(image):
    return cv2.GaussianBlur(image, (5, 5), 0)


def edge_detection(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=5)
    canny = cv2.Canny(gray, 100, 200)
    return sobelx, canny


def hough_transform(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)
    lines = cv2.HoughLines(edges, 1, np.pi/180, 200)
    if lines is not None:
        lines = lines.flatten()
    else:
        lines = np.zeros((1,))
    return lines


def sift_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    if descriptors is not None:
        descriptors = descriptors.flatten()
    else:
        descriptors = np.zeros((128,))
    return descriptors


def extract_features(image):
    features = []
    equalized = histogram_equalization(image)
    features.append(equalized.flatten())
    
    gray = grayscale_transformation(image)
    features.append(gray.flatten())
    
    smooth = image_smoothing(image)
    features.append(smooth.flatten())
    
    sobelx, canny = edge_detection(image)
    features.append(sobelx.flatten())
    features.append(canny.flatten())
    
    lines = hough_transform(image)
    features.append(lines.flatten())
    
    descriptors = sift_features(image)
    features.append(descriptors.flatten())
    
    # Pad or truncate each feature vector to a fixed length (e.g., 2000)
    max_length = 2000
    padded_features = []
    for feature in features:
        if feature.shape[0] > max_length:
            feature = feature[:max_length]
        else:
            feature = np.pad(feature, (0, max_length - feature.shape[0]), 'constant')
        padded_features.append(feature)
    
    return np.concatenate(padded_features)


# apply pca function
def apply_pca(features, n_components):
    pca = PCA(n_components=n_components)
    return pca.fit_transform(features)


In [148]:
# Load dataset
images = []
labels = []
label_map = {0: "Building", 1: "Forest", 2: "Glacier", 3: "Mountain", 4: "Sea", 5: "Street"}

for label, class_name in enumerate(label_map.values()):
    class_dir = dataset_paths[class_name]
    for file_name in os.listdir(class_dir):
        file_path = os.path.join(class_dir, file_name)
        image = cv2.imread(file_path)
        if image is not None:
            images.append(image)
            labels.append(label)

In [149]:
# Extract features for all images
features = np.array([extract_features(image) for image in images])

In [150]:
# Apply PCA to reduce dimensionality
reduced_features = apply_pca(features, n_components=10)

In [151]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(reduced_features, labels, test_size=0.2, random_state=42)

In [152]:
# Train a classification model
model = SVC()
model.fit(X_train, y_train)

SVC()

In [153]:
# Make predictions
y_pred = model.predict(X_test)

In [154]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 57.29%


In [155]:
# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=label_map.values()))

Accuracy: 0.572926596758818
Classification Report:
              precision    recall  f1-score   support

    Building       0.00      0.00      0.00       112
      Forest       0.75      0.95      0.84       539
     Glacier       0.29      0.02      0.04        94
    Mountain       0.33      0.01      0.02       107
         Sea       0.24      0.93      0.38        91
      Street       0.00      0.00      0.00       106

    accuracy                           0.57      1049
   macro avg       0.27      0.32      0.21      1049
weighted avg       0.47      0.57      0.47      1049



/home/aman/miniconda3/envs/tasks/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/aman/miniconda3/envs/tasks/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/aman/miniconda3/envs/tasks/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Second Approach
- Extract Colour Intensity from the Image and use that to train model
- Passing Image to PCA to extract Features and use that to train model
- Training the model

In [156]:
import os
import cv2
import numpy as np

# Define paths to datasets
dataset_paths = os.getcwd() + "/dataset_1/dataset_full"
dataset_paths = {
    "Building": os.path.join(dataset_paths, "Building"),
    "Forest": os.path.join(dataset_paths, "Forest"),
    "Glacier": os.path.join(dataset_paths, "Glacier"),
    "Mountain": os.path.join(dataset_paths, "Mountains"),
    "Sea": os.path.join(dataset_paths, "Sea"),
    "Street": os.path.join(dataset_paths, "Streets")
}


def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.resize(img, (128, 128))  # Resize image to a fixed size
            images.append(img)
    return images


def extract_color_intensity(image):
    # Calculate the mean color intensity for each channel (R, G, B)
    mean_intensity = image.mean(axis=(0, 1))
    return mean_intensity


def extract_features(image):
    features = []
    equalized = histogram_equalization(image)
    features.append(equalized.flatten())
    
    gray = grayscale_transformation(image)
    features.append(gray.flatten())
    
    smooth = image_smoothing(image)
    features.append(smooth.flatten())
    
    sobelx, canny = edge_detection(image)
    features.append(sobelx.flatten())
    features.append(canny.flatten())
    
    lines = hough_transform(image)
    features.append(lines.flatten())
    
    descriptors = sift_features(image)
    features.append(descriptors.flatten())
    
    # Pad or truncate each feature vector to a fixed length (e.g., 2000)
    max_length = 2000
    padded_features = []
    for feature in features:
        if feature.shape[0] > max_length:
            feature = feature[:max_length]
        else:
            feature = np.pad(feature, (0, max_length - feature.shape[0]), 'constant')
        padded_features.append(feature)
    
    return np.concatenate(padded_features)

# Load images from all categories
data = []
labels = []
color_intensity_features = []

for label, folder in dataset_paths.items():
    images = load_images_from_folder(folder)
    data.extend(images)
    labels.extend([label] * len(images))
    color_intensity_features.extend([extract_color_intensity(img) for img in images])



# for label, folder in dataset_paths.items():
#     images = load_images_from_folder(folder)
#     for img in images:
#         features = extract_features(img)
#         data.append(features)
#         color_intensity_features.append(extract_color_intensity(img))
#         labels.append(label)


data = np.array(data)
labels = np.array(labels)
color_intensity_features = np.array(color_intensity_features)

In [157]:
# Flatten the images to create feature vectors
data_flattened = data.reshape((data.shape[0], -1))

In [158]:
from sklearn.decomposition import PCA

# Apply PCA to reduce dimensionality
pca = PCA(n_components=30)  # Adjust the number of components based on your needs
data_reduced = pca.fit_transform(data_flattened)


In [159]:
# data_reduced

In [160]:
# import matplotlib.pyplot as plt
# import numpy as np

# PC_values = np.arange(pca.n_components_) + 1
# plt.plot(PC_values[0:40], pca.explained_variance_ratio_[0:40], 'o-', linewidth=0.1, color='blue')
# plt.title('Scree Plot')
# plt.xlabel('Principal Component')
# plt.ylabel('Variance Explained')
# plt.show()

In [161]:
# len(PC_values)

In [162]:
combined_features = np.hstack((data_reduced, color_intensity_features))

In [163]:
from sklearn.ensemble import RandomForestClassifier
#SVM
from sklearn.svm import SVC
# xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# Encode the labels
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(combined_features, labels_encoded, test_size=0.25, random_state=100)

In [164]:
# Train the model
clf = RandomForestClassifier(n_estimators=100, random_state=100, oob_score=True)
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 68.98%


In [165]:
# Train the model
clf = XGBClassifier()
clf.fit(X_train, y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 72.33%


In [166]:
# Train svm the model
clf = SVC()
clf.fit(X_train, y_train)
# Make predictions
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 71.88%


In [167]:
from sklearn.ensemble import StackingClassifier, VotingClassifier, BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [172]:
# Train svm the model
clf = StackingClassifier(estimators=[('xgb', XGBClassifier()),
                                    ('rf', RandomForestClassifier(random_state=42)),])
clf.fit(X_train, y_train)
# Make predictions
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 73.02%


In [169]:
# Assigning labels to the classes
label_map = {0: "Building", 1: "Forest", 2: "Glacier", 3: "Mountain", 4: "Sea", 5: "Street"}

In [173]:
# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=label_map.values()))

Accuracy: 0.7301829268292683
Classification Report:
              precision    recall  f1-score   support

    Building       0.55      0.41      0.47       117
      Forest       0.87      0.96      0.91       689
     Glacier       0.52      0.60      0.55       119
    Mountain       0.57      0.57      0.57       134
         Sea       0.43      0.32      0.37       135
      Street       0.64      0.52      0.57       118

    accuracy                           0.73      1312
   macro avg       0.60      0.56      0.57      1312
weighted avg       0.71      0.73      0.72      1312



In [176]:
# Define a function to make predictions on new images
def predict_image(image_path, clf, pca, le):
    img = cv2.imread(image_path)
    if img is not None:
        img = cv2.resize(img, (128, 128))  # Resize image to a fixed size
        img_flattened = img.reshape(1, -1)  # Flatten the image
        img_reduced = pca.transform(img_flattened)  # Apply PCA
        color_intensity = extract_color_intensity(img)  # Extract color intensity features
        combined_features = np.hstack((img_reduced, color_intensity.reshape(1, -1)))  # Combine features
        pred_encoded = clf.predict(combined_features)  # Predict
        pred_label = le.inverse_transform(pred_encoded)  # Decode label
        return pred_label[0]
    else:
        return None

# Example usage
example_image_path = "/home/aman/Personal/Task1/test_images/test1.jpg"
predicted_label = predict_image(example_image_path, clf, pca, le)
print(f"Predicted label: {predicted_label}")


Predicted label: Building


In [177]:
import pickle, os

model_path = os.path.join(os.getcwd() + "/models")
os.makedirs(model_path, exist_ok=True)

model_data = {
    'classifier': clf,
    'pca': pca,
    'label_encoder': le
}

with open(os.path.join(model_path, "models.pkl"), 'wb') as file:
    pickle.dump(model_data, file)
